In [1]:
import gym_examples
import gym
import numpy as np
# from stable_baselines3 import PPO
import sb3_contrib as sb3
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from gym.wrappers import FlattenObservation
from stable_baselines3.common.env_util import make_vec_env

D:\Softwares\Python3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sb3.__version__, gym.__version__

('1.7.0', '0.26.2')

# CNN ENV TESTING

In [4]:
env = gym.make("gym_examples/CNN-v0")

In [5]:
start_cnn = env.reset()
start_cnn

({'layer_type': 0,
  'layer_depth': 0,
  'filter_depth': 0,
  'filter_size': 0,
  'fc_size': 0,
  'is_start': 1,
  'pool_size_and_stride': 0},
 {'current_network': [{'layer_type': 'conv',
    'layer_depth': 0,
    'filter_depth': 0,
    'filter_size': 0,
    'fc_size': 0,
    'image_size': 28,
    'pool_size': 0,
    'pool_stride': 0,
    'is_start': 1}],
  'current_image_size': 28,
  'current_layer_depth': 0,
  'current_num_fc_layers': 0})

In [8]:
env.action_space

MultiDiscrete([3 9 5 4 5 2 4])

In [10]:
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros(shape=(state_space, action_space))
    return Qtable

In [12]:
def greedy_policy(Qtable, state):
    action = np.argmax(Qtable[state][:])
    return action

def epsilon_greedy_policy(Qtable, state, epsilon):
    random_num = random.uniform(0, 1)
    if random_num > epsilon:
        action = greedy_policy(Qtable, state)
    else:
        # TODO: need to change to a valid action
        action = env.action_space.sample()
    return action

In [19]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable, learning_rate, gamma, replay_memory_size):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        replay_memory = []
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
        # Reset the environment
        state = env.reset()
        print("Start:", state)
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            # TODO: !!!!!!!!!!!!!!!!!!!! SAMPLE VALID ACTION AND THEN DO epsilon greedy

            # Choose the action At using epsilon greedy policy
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            print("Action:",action)
            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, _, info = env.step(action)
            print("Next state:", new_state, " reward:", reward)
            replay_memory.append((new_state, action, reward))

            for memory in range(replay_memory_size):
                # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
                new_state, action, reward = random.choice(np.array(replay_memory))
                new_state = new_state.get("r2d2")
                new_state = new_state[0] * 5 + new_state[1]
                print(new_state)
                # handle termination here...
                Qtable[nestate][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            # If done, finish the episode
            if done:
                break
            # Our next state is the new state
            state = new_state
    return Qtable

In [24]:
state_space = env.observation_space

action_space = env.action_space

state_space, action_space

(Dict('fc_size': Discrete(5), 'filter_depth': Discrete(5), 'filter_size': Discrete(4), 'is_start': Discrete(2), 'layer_depth': Discrete(9), 'layer_type': Discrete(3), 'pool_size_and_stride': Discrete(4)),
 MultiDiscrete([3 9 5 4 5 2 4]))